# Object Detection in Google Colab with Fizyr Retinanet

Jupyter notebook providing steps to train a Keras/Tensorflow model for object detection with custom dataset.

It runs in Google Colab using [Fizyr implementation](https://github.com/fizyr/keras-retinanet) of RetinaNet in Keras.

Requirements are only dataset images and annotations file made in [LabelImg](https://github.com/tzutalin/labelImg).

Colab Runtime type: Python3, GPU enabled.

# Environment Setup
Download and install in Colab required packages and import libraries.

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
%cd keras-retinanet/

!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import os
import shutil
import zipfile
import urllib
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

# Making Dataset

We will be using HyperLabel to create our dataset. You can watch the following tutorial video about how to use HyperLabel and tag images with it.

https://www.youtube.com/embed/R56Ck3tElIs

Once all of your images are tagged, its time to export the labels and data in Pascal VOC format. Goto the `review` tab/section and click on the export button. From the export menu, choose `Object Detection` and select Pascal VOC and click on export. It will take some time to export the data, once exported you should be able to see multiple folders, but we are interested in two i.e., `Annotations` and `JPEGImages` folder.

Now created a new folder, name it `dataset`. Copy all the contents of `Annotations` folder and `JPEGImages` folder inside `dataset` folder and then create a zip archive of this folder.

Upload this zip archive to your Google drive and get the ID. To locate the File ID, right click on the name of the file, choose the Get Shareable Link option, and turn on Link Sharing if needed (you can turn it off later). You will see the link with a combination of numbers and letters at the end, and what you see after id =   is the File ID.

https://drive.google.com/open?id=***ThisIsFileID*** 

If your file is already open in a browser, you can obtain File ID from its link:

https://docs.google.com/spreadsheets/d/***ThisIsFileID***/edit#gid=123456789 

In [ ]:
! git clone https://github.com/omrimanor7/TrafficSignDetection2021.git

Cloning into 'TrafficSignDetection2021'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 2195 (delta 8), reused 26 (delta 4), pack-reused 2161
Receiving objects: 100% (2195/2195), 1.60 GiB | 25.04 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (2131/2131), done.


In [ ]:
CLASSES_FILE = "/content/keras-retinanet/TrafficSignDetection2021/FullIJCNN2013/classes.csv"
ANNOTATIONS_FILE = "/content/keras-retinanet/TrafficSignDetection2021/FullIJCNN2013/annotations.csv"

# Training Model

Download pretrained model and run training.

In the next cell choose one option:

1.   download Fizyr Resnet50 pretrained model
2.   download your custom pretrained model, to continue previous training epochs

In the last cell optionally export trained model to Google Drive.


In [ ]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
# URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
# URL_MODEL = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####
drive.mount('/content/gdrive')
DRIVE_MODEL = '/content/gdrive/My Drive/EE Final Project/ResNetModels/resnet50_csv_01.h5'
shutil.copy(DRIVE_MODEL, './snapshots/resnet50_csv_01.h5')

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Mounted at /content/gdrive
Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [ ]:
# !keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 1 --steps 900 --epochs 1 csv {ANNOTATIONS_FILE} {CLASSES_FILE}

In [ ]:
SNAPSHOT = './snapshots/resnet50_csv_01.h5'

In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --snapshot {SNAPSHOT} --batch-size 1 --steps 300 --epochs 1 csv {ANNOTATIONS_FILE} {CLASSES_FILE}


In [ ]:
#### OPTIONAL: EXPORT TRAINED MODEL TO DRIVE ####
drive.mount('/content/gdrive')
COLAB_MODEL = './snapshots/resnet50_csv_01.h5'
DRIVE_DIR = '/content/gdrive/My Drive/EE Final Project/ResNetModels/'
# shutil.copy(COLAB_MODEL, DRIVE_DIR)
DRIVE_MODEL = DRIVE_DIR + 'resnet50_csv_01.h5'
shutil.copy(DRIVE_MODEL, COLAB_MODEL)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'./snapshots/resnet50_csv_01.h5'

In [ ]:
import sys
import runpy
sys.argv = f"keras_retinanet/bin/train.py --freeze-backbone --random-transform --snapshot {SNAPSHOT} --batch-size 1 --steps 700 --epochs 1 csv {ANNOTATIONS_FILE} {CLASSES_FILE}".split()
import subprocess

while True:
  subprocess.call(sys.argv)
  shutil.copy(COLAB_MODEL, DRIVE_DIR)

# Inference
Run inference with uploaded image on trained model.

In [ ]:
THRES_SCORE = 0.6

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
tf.compat.v1.keras.backend.set_session(get_session())

In [ ]:
model_path = DRIVE_MODEL
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

/content/gdrive/My Drive/EE Final Project/ResNetModels/resnet50_csv_01.h5


In [ ]:
def img_inference(img_path):
  image = read_image_bgr(img_infer)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      
      # print(label)
      
      # scores are sorted so we can break
      if score < THRES_SCORE:
          break

      color = label_color(label)

      b = box.astype(int)
      draw_box(draw, b, color=color)

      caption = "{} {:.3f}".format(labels_to_names[label], score)
      print(caption)
      draw_caption(draw, b, caption)

  plt.figure(figsize=(20, 20))
  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [ ]:
# GTSDB


# uploaded = files.upload()
# img_infer = list(uploaded)[0]
# img_inference(img_infer)

test_imgs_indecies = np.random.randint(0, 900, 10)
for i in test_imgs_indecies:
  img_infer = '/content/keras-retinanet/TrafficSignDetection2021/FullIJCNN2013/' + str(i).zfill(5)  + '.ppm'
  print('Running inference on: ' + img_infer)
  img_inference(img_infer)




In [ ]:
# chinese


# uploaded = files.upload()
# img_infer = list(uploaded)[0]
# img_inference(img_infer)

test_imgs_indecies = np.random.randint(1, 100, 10)
for i in test_imgs_indecies:
  img_infer = '/content/gdrive/MyDrive/EE Final Project/chineseDataset/test_image/test (' + str(i)  + ').bmp'
  print('Running inference on: ' + img_infer)
  img_inference(img_infer)


